# Book Recommendation System

This Streamlit app provides personalized book recommendations based on either user history or book similarity. Users can input a book title or user ID to receive recommendations from two models (SVD and KNN). The app includes:

- **Interactive Tables**: Displays recommended books with key details like title, author, and predicted rating.
- **Charts**: Visualizes book recommendations with bar charts using Plotly.
- **Favorites**: Users can save books or users to their favorites for quick access.
- **Export Options**: Recommendations can be exported as CSV or Excel files for further analysis.
- **Session Management**: Keeps track of favorites and current recommendations using session state.

### Features:
1. **User-based Recommendations**: Input a user ID to get book recommendations based on user history.
2. **Book-based Recommendations**: Input a book title to find similar books.
3. **Interactive UI**: Use tabs for visualizations and detailed tables.
4. **Export**: Download recommendations in CSV or Excel formats.
5. **Favorites**: Save books or users to a personal favorites list.

### Technologies Used:
- **Streamlit**: For building the interactive UI.
- **Plotly**: For creating interactive charts.
- **Pandas**: For data manipulation.
- **st_aggrid**: For interactive tables.

### Prerequisites:
To run the app, ensure you have the following installed:
```bash
pi install streamlit-aggrid


In [ ]:
import streamlit as st
import requests
import pandas as pd
import plotly.graph_objects as go
from st_aggrid import AgGrid, GridOptionsBuilder
from st_aggrid.shared import GridUpdateMode
from typing import Dict, List

# Constants
API_URL = "http://localhost:8000"  # URL of the API for recommendations

# Custom CSS for styling the Streamlit app
st.markdown("""
<style>
    .reportview-container .main .block-container {
        max-width: 1200px;
        padding-top: 2rem;
        padding-bottom: 2rem;
    }
    .stDataFrame {
        border: 1px solid #ddd;
        border-radius: 5px;
        padding: 1rem;
    }
    .styled-table {
        background-color: #f5f5f5;
        padding: 1rem;
        border-radius: 5px;
    }
    .metric-card {
        background-color: #ffffff;
        border: 1px solid #ddd;
        border-radius: 5px;
        padding: 1rem;
        text-align: center;
    }
    .metric-value {
        font-size: 24px;
        font-weight: bold;
        color: #0066cc;
    }
</style>
""", unsafe_allow_html=True)  # Apply custom CSS for better UI

# Cache configuration for API requests to optimize performance
@st.cache_data(ttl=3600)  # Cache data for 1 hour
def get_recommendations(endpoint: str, params: Dict) -> Dict:
    """Get recommendations from the API based on the endpoint and parameters"""
    try:
        response = requests.get(f"{API_URL}/{endpoint}", params=params)
        response.raise_for_status()  # Raise an error if the request failed
        return response.json()  # Return the response as a JSON object
    except requests.exceptions.RequestException as e:
        st.error(f"Error fetching data: {str(e)}")  # Display error message if API call fails
        return {}

def create_interactive_table(data: List[Dict], title: str) -> None:
    """Create an interactive table using AgGrid"""
    df = pd.DataFrame(data)  # Convert the list of dictionaries to a pandas DataFrame
    gb = GridOptionsBuilder.from_dataframe(df)  # Build grid options from the DataFrame
    gb.configure_pagination(paginationAutoPageSize=False, paginationPageSize=5)  # Pagination settings
    gb.configure_column("Title", headerName="Book Title", width=250)  # Column for book title
    gb.configure_column("Author", width=150)  # Column for author
    gb.configure_column("Year", width=100)  # Column for publication year
    gb.configure_column("Publisher", width=150)  # Column for publisher
    
    # Check if columns for predicted rating and similarity exist and format them
    if "Predicted_Rating" in df.columns:
        gb.configure_column("Predicted_Rating", 
                            headerName="Rating", 
                            width=120, 
                            valueFormatter="'⭐' + data.Predicted_Rating.toFixed(2)")  # Format predicted rating
    
    if "Similarity" in df.columns:
        gb.configure_column("Similarity", 
                            headerName="Match %", 
                            width=120, 
                            valueFormatter="data.Similarity.toFixed(2) + '%'")  # Format similarity percentage
    
    gb.configure_selection('single')  # Allow single row selection
    grid_options = gb.build()  # Build the grid options

    st.subheader(title)  # Display the table title
    grid_response = AgGrid(
        df, 
        gridOptions=grid_options, 
        update_mode=GridUpdateMode.SELECTION_CHANGED, 
        fit_columns_on_grid_load=True, 
        allow_unsafe_jscode=True, 
        theme='material'  # Use the material theme for the grid
    )
    
    # If a row is selected, display its details
    if grid_response['selected_rows']:
        selected = grid_response['selected_rows'][0]
        st.markdown("### 📖 Selected Book Details")
        st.write(f"**Title**: {selected['Title']}")
        st.write(f"**Author**: {selected['Author']}")
        st.write(f"**Year**: {selected['Year']}")
        st.write(f"**Publisher**: {selected['Publisher']}")
        if "Predicted_Rating" in selected:
            st.write(f"**Predicted Rating**: ⭐ {selected['Predicted_Rating']:.2f}")
        if "Similarity" in selected:
            st.write(f"**Similarity Score**: 🎯 {selected['Similarity']:.2f}%")

def create_chart(df: pd.DataFrame, x: str, y: str, title: str) -> go.Figure:
    """Create an interactive plotly chart"""
    fig = go.Figure(go.Bar(
        x=df[x],
        y=df[y],
        marker_color='rgb(158,202,225)',  # Bar color
        text=df[y].round(2),  # Display rounded values as text on the bars
        textposition='auto'  # Position the text inside the bars
    ))
    
    # Update layout for the chart
    fig.update_layout(
        title=title,
        title_x=0.5,
        xaxis_title="Books",
        yaxis_title="Score",
        template="plotly_white",
        height=500,
        showlegend=False,
        xaxis_tickangle=-45  # Rotate x-axis labels for better readability
    )
    
    return fig  # Return the figure object

def display_user_recommendations(recommendations: Dict) -> None:
    """Display user-based recommendations with visualizations and tables"""
    if not recommendations:
        return
    
    # Create tabs for visualizations and detailed tables
    tab1, tab2 = st.tabs(["📊 Visualizations", "📋 Detailed Tables"])
    
    with tab1:
        col1, col2 = st.columns(2)
        with col1:
            df_svd = pd.DataFrame(recommendations['svd'])  # Convert SVD data to DataFrame
            fig_svd = create_chart(df_svd, 'Title', 'Predicted_Rating', 'SVD Model Recommendations')
            st.plotly_chart(fig_svd, use_container_width=True)  # Display the SVD chart
            
        with col2:
            df_knn = pd.DataFrame(recommendations['knn'])  # Convert KNN data to DataFrame
            fig_knn = create_chart(df_knn, 'Title', 'Predicted_Rating', 'KNN Model Recommendations')
            st.plotly_chart(fig_knn, use_container_width=True)  # Display the KNN chart
    
    with tab2:
        create_interactive_table(recommendations['svd'], "📚 SVD Recommendations")  # Display SVD table
        create_interactive_table(recommendations['knn'], "📚 KNN Recommendations")  # Display KNN table

def display_book_recommendations(recommendations: Dict) -> None:
    """Display book-based recommendations with visualizations and tables"""
    if not recommendations:
        return
    
    # Create tabs for visualizations and detailed tables
    tab1, tab2 = st.tabs(["📊 Visualization", "📋 Detailed Table"])
    
    with tab1:
        df = pd.DataFrame(recommendations['recommendations'])  # Convert recommendations to DataFrame
        fig = create_chart(df, 'Title', 'Similarity', 'Similar Books Comparison')
        st.plotly_chart(fig, use_container_width=True)  # Display the chart
    
    with tab2:
        create_interactive_table(recommendations['recommendations'], "📚 Similar Books")  # Display detailed table

def save_to_favorites(item: Dict, item_type: str) -> None:
    """Save item to favorites in session state"""
    if 'favorites' not in st.session_state:
        st.session_state.favorites = {'books': [], 'users': []}  # Initialize favorites if not present
    
    favorites = st.session_state.favorites[item_type + 's']  # Select the correct favorites list
    if item not in favorites:
        favorites.append(item)  # Add item to favorites if not already present

def show_favorites() -> None:
    """Display favorites in the sidebar"""
    if 'favorites' in st.session_state:
        st.sidebar.markdown("### ⭐ Favorites")  # Show favorites section in sidebar
        for item_type in ['books', 'users']:
            if st.session_state.favorites[item_type]:
                st.sidebar.markdown(f"#### Favorite {item_type.capitalize()}")
                for item in st.session_state.favorites[item_type]:
                    st.sidebar.markdown(f"- {item['Title'] if item_type == 'books' else f'User ID: {item}'}")

def export_recommendations(recommendations: Dict, export_type: str) -> None:
    """Export recommendations to CSV or Excel"""
    if not recommendations:
        return
        
    # Export to Excel
    if export_type == 'excel':
        with pd.ExcelWriter('recommendations.xlsx', engine='openpyxl') as writer:
            if 'svd' in recommendations:
                pd.DataFrame(recommendations['svd']).to_excel(writer, sheet_name='SVD Recommendations', index=False)
                pd.DataFrame(recommendations['knn']).to_excel(writer, sheet_name='KNN Recommendations', index=False)
            else:
                pd.DataFrame(recommendations['recommendations']).to_excel(writer, sheet_name='Similar Books', index=False)
        
        with open('recommendations.xlsx', 'rb') as f:
            st.sidebar.download_button(
                label="📥 Download Excel Report",
                data=f.read(),
                file_name='recommendations.xlsx',
                mime='application/vnd.ms-excel'
            )
    # Export to CSV
    else:
        csv_data = pd.DataFrame(recommendations['recommendations']).to_csv(index=False)
        st.sidebar.download_button(
            label="📥 Download CSV Report",
            data=csv_data,
            file_name='recommendations.csv',
            mime='text/csv'
        )

def main():
    """Main function to run the Streamlit app"""
    st.title("📚 Book Recommendation System")  # Set the title of the app
    st.markdown("""
    Get personalized book recommendations based on user history or find similar books to your favorites!
    Choose a recommendation type from the sidebar to begin.
    """)
    
    st.sidebar.title("⚙️ Configuration")  # Sidebar configuration
    recommendation_type = st.sidebar.radio(
        "Choose Recommendation Type",
        ["User-based", "Book-based"]
    )
    
    num_recommendations = st.sidebar.slider(
        "Number of Recommendations",
        min_value=5,
        max_value=20,
        value=10,
        step=5
    )
    
    # User-based recommendations
    if recommendation_type == "User-based":
        st.header("👤 User-based Recommendations")
        
        user_id = st.number_input("Enter User ID", min_value=1, value=276725)
        if st.button("🔍 Get Recommendations"):
            with st.spinner("Getting recommendations..."):
                recommendations = get_recommendations("user/" + str(user_id), {'num_recommendations': num_recommendations})
                if recommendations:
                    st.session_state.current_recommendations = recommendations
                    st.success("✅ Recommendations generated successfully!")
                    if st.button("💾 Save to Favorites"):
                        save_to_favorites({'Title': f'User {user_id}', 'ID': user_id}, 'user')
                    display_user_recommendations(recommendations)
    
    # Book-based recommendations
    elif recommendation_type == "Book-based":
        st.header("📚 Book-based Recommendations")
        
        book_title = st.text_input("Enter Book Title", "Harry Potter")
        if st.button("🔍 Get Recommendations"):
            with st.spinner("Getting recommendations..."):
                recommendations = get_recommendations("book", {'book_title': book_title, 'num_recommendations': num_recommendations})
                if recommendations:
                    st.session_state.current_recommendations = recommendations
                    st.success("✅ Recommendations generated successfully!")
                    if st.button("💾 Save to Favorites"):
                        save_to_favorites({'Title': book_title}, 'book')
                    display_book_recommendations(recommendations)
    
    show_favorites()  # Display favorites in sidebar
    export_recommendations(st.session_state.get('current_recommendations', {}), export_type='csv')  # Export recommendations

if __name__ == "__main__":
    main()  # Run the main function

### Explanation of the Command `!streamlit run app.py`
### Steps Involved When Running the Command:
1. **Streamlit starts a local server**: The command launches a local web server to host the application.
2. **Runs the `app.py` script**: Streamlit executes the Python code inside the `app.py` file.
3. **Displays the app in a browser**: Once the server is running, Streamlit automatically opens the app in your default web browser (usually at `http://localhost:8501`).

This command is commonly used for testing and deploying Streamlit apps locally before sharing or deploying them to a cloud service.

In [ ]:
!streamlit run app.py